In [4]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
provider = SaverProvider() 

In [5]:
from Scratch import metadata_loader
import pandas as pd

df = metadata_loader(True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]

# df = df[df['backend_name']=='ibm_sherbrooke']
# df = df[df['rounds'] == 50]
# df = df[df['xbasis'] == False]
# df = df[df['logical'] == "0"]

# df = df.dropna(subset=['sampled_state'])
# df = df[pd.isna(df['double_msmt'])]

df = df[::-1]
df

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets,num_qubit,double_msmt,xbasis,path_info


In [4]:
from time import sleep

OVERWRITE = True
IGNORE_RUNNING = False

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=IGNORE_RUNNING, overwrite=OVERWRITE)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            # break  
            sleep(5)


  0%|          | 0/120 [00:00<?, ?it/s]

17:15:02 Warning: Overwriting job ID crcjcc9622j0008m8t10 in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 79
17:15:50 Warning: Overwriting job ID crcjcf169nf0008fe980 in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 79
17:16:40 Warning: Overwriting job ID crcjd5wdqdn000833xng in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 79
17:17:30 Warning: Overwri

# CLEANUP: Check if jobs have files

In [6]:
md = metadata_loader(True)

for job_id in tqdm(md.job_id):

    filename = provider.job_local_filename(job_id)
    status = md.loc[md['job_id'] == job_id, 'job_status'].values[0]

    if filename == None and status not in ['JobStatus.ERROR', 'JobStatus.CANCELLED']:
        provider.retrieve_job(job_id)

    # if filename == None and status == 'JobStatus.DONE':
    #     provider.retrieve_job(job_id)

  0%|          | 0/6467 [00:00<?, ?it/s]

# update the metadata with the execution date

In [24]:
from Scratch import update_metadata, find_and_create_scratch    
import os

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.ERROR', 'JobStatus.CANCELLED'])]

start_idx = 0
while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            # check if the entry has NaN for its execution_date
            if pd.isna(df.loc[df['job_id'] == job_id, 'execution_date'].values[0]):
                ibm_job = provider.retrieve_job(job_id, _ignore_running=False)
                additional_dict = {"execution_date": str(ibm_job.result().date)}
                update_metadata(ibm_job, "None_Backend", additional_dict)

                start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        # if 'JSON' in str(type(e)):
        else:
            # remove the file from the currect job_id
            path = find_and_create_scratch()
            jobs_path = os.path.join(path, 'jobs')
            for filename in os.listdir(jobs_path):
                if job_id in filename:
                    os.remove(os.path.join(jobs_path, filename))
                    print(f'\nJSONDecodeError occurred at job index {start_idx}, removed file, retrying...')

        # else:
        #     print(f'An unexpected error occurred: {e}')
        #     break  
            

  0%|          | 0/4349 [00:00<?, ?it/s]